In [1]:
import sys
import os
import cv2 as cv
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
from statistics import mode
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import f1_score
from heapq import * 
import time
import copy

#Pytorch
import torch
import torchvision
# import torchvision.transforms as transforms
from torchvision import datasets, models, transforms
from skimage import io
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import LambdaLR
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from sklearn.model_selection import KFold
from scipy import stats
from sklearn.metrics import f1_score, confusion_matrix

import pytorch_model_summary as pms
from torchviz import make_dot
np.random.seed(42)

# import tensorflow as tf
NUM_CLASS = 2
WINDOW = 5
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
class MyDataset(Dataset):
    def __init__(self, root_dir=None, transform=None):
        path = os.path.join(root_dir,'labels.csv')
        self.ylabels = pd.read_csv(path)
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.ylabels)

    def __getitem__(self, idx):
        seq_len = 5
        if torch.is_tensor(idx):
            idx = idx.tolist()
 
        label = self.ylabels.iloc[idx,1]
        label_str = str(label) + '/'
        img_folder = os.path.join(self.root_dir, label_str)
        
        img_arr = np.zeros((5, 1, 64, 64))
        for i in range (5):
            img_name = img_folder + str(idx) + '-' + str(i) + '.jpg'
            image = io.imread(img_name)
            if self.transform:
                image = self.transform(image)
            img_arr[i][0] = image
            
        label_1hot = np.zeros(NUM_CLASS)
        label_1hot[label] = 1.0
        label_1hot = torch.Tensor(label_1hot)
#         label = np.array([label])
        label = torch.LongTensor(np.array([label]))
        sample = {'images': img_arr, 'label': label_1hot}
        return sample

In [3]:
class Reshape1(torch.nn.Module):
    def forward(self, x):
        batch_size = torch.Size([1,5,1,64,64])
        return x.view(batch_size, -1)
    
class Reshape2(torch.nn.Module):
    def forward(self, x):
        batch_size = torch.Size([1,1,64,64])
        return x.view(batch_size, -1)

def build_encoder():
    encoder = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=192, kernel_size=3),
        nn.BatchNorm2d(num_features=192),
        nn.ReLU(),
        nn.Conv2d(in_channels=192, out_channels=192, kernel_size=1),
        nn.ReLU(),
        
        nn.Conv2d(in_channels=192, out_channels=192, kernel_size=3, stride=2),
        nn.BatchNorm2d(num_features=192),
        nn.ReLU(),
        nn.Dropout(p=0.5),
        
        nn.Conv2d(in_channels=192, out_channels=96, kernel_size=3),
        nn.BatchNorm2d(num_features=96),
        nn.ReLU(),
        nn.Conv2d(in_channels=96, out_channels=96, kernel_size=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=96, out_channels=96, kernel_size=1),
        nn.ReLU(),
        
        nn.MaxPool2d(kernel_size=3, stride=2),
        nn.Dropout(p=0.5),
        
        nn.Conv2d(in_channels=96, out_channels=32, kernel_size=3),
        nn.BatchNorm2d(num_features=32),
        nn.ReLU(),
        nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3),
        nn.BatchNorm2d(num_features=32),
        nn.ReLU(),
        nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=2),
        nn.BatchNorm2d(num_features=32),
        nn.ReLU()
    )
    return encoder

def build_decoder():
    decoder = nn.Sequential(
        nn.Conv2d(in_channels=32, out_channels=NUM_CLASS, kernel_size=1),
        nn.AdaptiveAvgPool2d((1,1)),
        nn.Softmax(dim=1)
    )
    return decoder

In [4]:
class Net(nn.Module):
    def __init__(self, encoder, decoder):
        super(Net, self).__init__()
        self.in_encoder = encoder
        self.in_decoder = decoder
        
    def forward(self, x):
        tmp = self.in_encoder(x)
        y = self.in_decoder(tmp)
        return y
    def encoder(self, x):
        return self.in_encoder(x)

In [5]:
encoder = build_encoder()
decoder = build_decoder()
net = Net(encoder, decoder)
pms.summary(net, 
    torch.zeros((1,1,64,64)), 
    batch_size=1, 
    show_hierarchical=True, 
    print_summary=True)

-----------------------------------------------------------------------------
           Layer (type)         Output Shape         Param #     Tr. Param #
               Conv2d-1     [1, 192, 62, 62]           1,920           1,920
          BatchNorm2d-2     [1, 192, 62, 62]             384             384
                 ReLU-3     [1, 192, 62, 62]               0               0
               Conv2d-4     [1, 192, 62, 62]          37,056          37,056
                 ReLU-5     [1, 192, 62, 62]               0               0
               Conv2d-6     [1, 192, 30, 30]         331,968         331,968
          BatchNorm2d-7     [1, 192, 30, 30]             384             384
                 ReLU-8     [1, 192, 30, 30]               0               0
              Dropout-9     [1, 192, 30, 30]               0               0
              Conv2d-10      [1, 96, 28, 28]         165,984         165,984
         BatchNorm2d-11      [1, 96, 28, 28]             192             19

'-----------------------------------------------------------------------------\n           Layer (type)         Output Shape         Param #     Tr. Param #\n=============================================================================\n               Conv2d-1     [1, 192, 62, 62]           1,920           1,920\n          BatchNorm2d-2     [1, 192, 62, 62]             384             384\n                 ReLU-3     [1, 192, 62, 62]               0               0\n               Conv2d-4     [1, 192, 62, 62]          37,056          37,056\n                 ReLU-5     [1, 192, 62, 62]               0               0\n               Conv2d-6     [1, 192, 30, 30]         331,968         331,968\n          BatchNorm2d-7     [1, 192, 30, 30]             384             384\n                 ReLU-8     [1, 192, 30, 30]               0               0\n              Dropout-9     [1, 192, 30, 30]               0               0\n              Conv2d-10      [1, 96, 28, 28]         165,984 

In [5]:
def check_validation(model, valloader, loss_function):
    # Evaluationfor this fold
    correct, total = 0, 0
    current_loss = 0.0
    with torch.no_grad():
        # Init the neural network
        encoder = build_encoder()
        decoder = build_decoder()
        trained_net = Net(encoder, decoder)
        trained_net.load_state_dict(model)
        trained_net.to(device)

        # Iterate over the test data and generate predictions
        for i, data in enumerate(valloader, 0):

            # Get inputs
            inputs = data['images']
            targets = data['label']

            outputs = None
            for index, xbatch in enumerate(inputs):

                xbatch = xbatch.float().to(device)
                output = sliding_window(xbatch, trained_net)

                # Perform forward pass
#                 output = trained_net(xbatch)
                if index == 0:
                    outputs = output
                else:
                    outputs = torch.vstack([outputs, output])

            # Compute loss
            ybatches = targets.to(device)
            outputs = torch.reshape(outputs, (ybatches.shape[0],NUM_CLASS))
            ybatches.squeeze_(1)
            loss = loss_function(outputs, ybatches)
            current_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            _, truth = torch.max(ybatches, 1)
            total += targets.size(0)
            correct += (predicted == truth).sum().item()
    val_loss = current_loss/(i+1)
    val_acc = 100.0 * correct / total
    return val_loss, val_acc

In [6]:
def sliding_window(x, network):         
    outputs = network(x)
    outputs = torch.reshape(outputs,(-1,2))
    final_out = torch.mean(outputs, 0)
    return final_out

In [9]:
def train():
    #---------------Config---------------
    torch.manual_seed(42)
    num_epochs = 50
    k_folds = 5
    patience = 10
    batch_size = 32
    #---------------Config---------------
    
    # Preparation
    weight = torch.tensor([1,1]).to(device)
    
    loss_function = nn.BCELoss(weight=weight)

    transformer = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5)),
    ])

    train_data_dir = './Prepared_Data/Train/'
    train_dataset = MyDataset(root_dir=train_data_dir, transform = transformer)

    val_data_dir = './Prepared_Data/Validation/'
    val_dataset = MyDataset(root_dir=val_data_dir, transform = transformer)

    dataset = ConcatDataset([train_dataset, val_dataset])

    kfold = KFold(n_splits=k_folds, shuffle=True)
    
    results = {}

    # K-fold Cross Validation model evaluation
    for fold, (train_ids, val_ids) in enumerate(kfold.split(dataset)):
        training_details = []
        best_model = None
        print(len(train_ids))
        print(len(val_ids))

        # Print
        print(f'FOLD {fold}')
        print('--------------------------------')

        # Sample elements randomly from a given list of ids, no replacement.
        train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
        val_subsampler = torch.utils.data.SubsetRandomSampler(val_ids)

        # Define data loaders for training and testing data in this fold
        trainloader = torch.utils.data.DataLoader(
                        dataset,
                        batch_size=batch_size, sampler=train_subsampler)
        valloader = torch.utils.data.DataLoader(
                        dataset,
                        batch_size=batch_size, sampler=val_subsampler)

        # Init the neural network
        encoder = build_encoder()
        decoder = build_decoder()
        network = Net(encoder, decoder)
        network.to(device) #use GPU
        use_gpu = True

        # Initialize optimizer
        optimizer = optim.Adam(network.parameters(), lr=0.001)
        max_train_acc, max_val_acc = 0, 0
        early_stop_count = 0
        # Run the training loop for defined number of epochs
        for epoch in range(0, num_epochs):

            correct, total = 0, 0
            # Print epoch
            print(f'Starting epoch {epoch+1}')
            # Set current loss value
            current_loss = 0.0
            epoch_loss = 0.0
            # Iterate over the DataLoader for training data
            for i, data in enumerate(trainloader, 0):
            # Starting 1 cycle of mini-batch

                # Get inputs
                inputs = data['images']
                targets = data['label']

                outputs = None
                for index, xbatch in enumerate(inputs):
                    # Zero the gradients
                    optimizer.zero_grad()

                    xbatch = xbatch.float().to(device)
                    output = sliding_window(xbatch, network)

                    # Perform forward pass
                    if index == 0:
                        outputs = output
                    else:
                        outputs = torch.vstack([outputs, output])

                # Compute loss
                ybatches = targets.to(device)
                outputs = torch.reshape(outputs, (ybatches.shape[0],NUM_CLASS))
                loss = loss_function(outputs, ybatches)

                # Perform backward pass
                loss.backward()

                # Perform optimization
                optimizer.step()

                # Print statistics
                current_loss += loss.item()
                epoch_loss += loss.item()

                # Compute Accuracy
                _, predicted = torch.max(outputs, 1)
                _, truth = torch.max(ybatches, 1)
                total += targets.size(0)
                correct += (predicted == truth).sum().item()

            epoch_loss = epoch_loss/(i+1)
            epoch_acc = 100.0 * correct / total
            print('Train Loss for Epoch %d: %.4f ' % (epoch, epoch_loss))
            print('Train Accuracy for Epoch %d: %.2f %%' % (epoch, epoch_acc))
            val_loss, val_acc = check_validation(network.state_dict(), valloader, loss_function)
            print('Val Loss for Epoch %d: %.4f ' % (epoch, val_loss))
            print('Val Accuracy for Epoch %d: %.2f %%' % (epoch, val_acc))
            print('--------------------------------')

            training_details.append([epoch_acc, epoch_loss, val_acc, val_loss])

            stop_improving = False
            if val_acc > max_val_acc:
                max_val_acc = val_acc
                early_stop_count = 0
            else:
                early_stop_count += 1
                stop_improving = True
                print("Early Stop Count Increased")

            if epoch_acc > max_train_acc:
                max_train_acc = copy.deepcopy(epoch_acc)
                best_model = copy.deepcopy(network.state_dict())
                print("Found better model...saving")

            if early_stop_count >= patience:
                print("Early stopping at Epoch", epoch)
                break


        # Process is complete.
        print('Training process has finished. Saving trained model.')

        # Print about testing
        print('Starting testing')

        # Saving the model
        save_path = f'./models/model-fold-{fold}.pth'
        save_path_csv = f'./models/model-fold-{fold}.csv'
        torch.save(best_model, save_path)

        training_details = np.array(training_details)
        df = {'epoch acc': training_details[:,0], 
              'epoch loss': training_details[:,1], 
              'val acc': training_details[:,2], 
              'val loss': training_details[:,3], }
        pd.DataFrame(df).to_csv(save_path_csv)

        # Evaluationfor this fold
        correct, total = 0, 0
        with torch.no_grad():
            val_loss, val_acc = check_validation(torch.load(save_path), valloader, loss_function)
            # Print accuracy
            print('Accuracy for fold %d: %.2f %%' % (fold, val_acc))
            print('--------------------------------')
            results[fold] = val_acc

    # Print fold results
    print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
    print('--------------------------------')
    sum = 0.0
    for key, value in results.items():
        print(f'Fold {key}: {value} %')
        sum += value
    print(f'Average: {sum/len(results.items())} %')

In [8]:
train()

FOLD 0
--------------------------------
Starting epoch 1
Train Loss for Epoch 0: 0.6937 
Train Accuracy for Epoch 0: 50.56 %
Val Loss for Epoch 0: 0.6870 
Val Accuracy for Epoch 0: 50.00 %
--------------------------------
Found better model...saving
Starting epoch 2
Train Loss for Epoch 1: 0.6831 
Train Accuracy for Epoch 1: 51.40 %
Val Loss for Epoch 1: 0.6703 
Val Accuracy for Epoch 1: 77.78 %
--------------------------------
Found better model...saving
Starting epoch 3
Train Loss for Epoch 2: 0.6668 
Train Accuracy for Epoch 2: 80.45 %
Val Loss for Epoch 2: 0.6492 
Val Accuracy for Epoch 2: 91.11 %
--------------------------------
Found better model...saving
Starting epoch 4
Train Loss for Epoch 3: 0.6478 
Train Accuracy for Epoch 3: 85.47 %
Val Loss for Epoch 3: 0.6313 
Val Accuracy for Epoch 3: 87.78 %
--------------------------------
Early Stop Count Increased
Found better model...saving
Starting epoch 5
Train Loss for Epoch 4: 0.6225 
Train Accuracy for Epoch 4: 87.99 %
Val Loss

Val Loss for Epoch 9: 0.4855 
Val Accuracy for Epoch 9: 83.33 %
--------------------------------
Early Stop Count Increased
Found better model...saving
Starting epoch 11
Train Loss for Epoch 10: 0.3891 
Train Accuracy for Epoch 10: 94.41 %
Val Loss for Epoch 10: 0.4377 
Val Accuracy for Epoch 10: 86.67 %
--------------------------------
Starting epoch 12
Train Loss for Epoch 11: 0.3521 
Train Accuracy for Epoch 11: 94.69 %
Val Loss for Epoch 11: 0.4493 
Val Accuracy for Epoch 11: 83.33 %
--------------------------------
Early Stop Count Increased
Starting epoch 13
Train Loss for Epoch 12: 0.3250 
Train Accuracy for Epoch 12: 94.69 %
Val Loss for Epoch 12: 0.3993 
Val Accuracy for Epoch 12: 86.67 %
--------------------------------
Early Stop Count Increased
Starting epoch 14
Train Loss for Epoch 13: 0.2862 
Train Accuracy for Epoch 13: 95.81 %
Val Loss for Epoch 13: 0.4382 
Val Accuracy for Epoch 13: 81.11 %
--------------------------------
Early Stop Count Increased
Found better model.

Accuracy for fold 2: 88.89 %
--------------------------------
FOLD 3
--------------------------------
Starting epoch 1
Train Loss for Epoch 0: 0.6932 
Train Accuracy for Epoch 0: 50.14 %
Val Loss for Epoch 0: 0.6868 
Val Accuracy for Epoch 0: 49.44 %
--------------------------------
Found better model...saving
Starting epoch 2
Train Loss for Epoch 1: 0.6828 
Train Accuracy for Epoch 1: 53.48 %
Val Loss for Epoch 1: 0.6723 
Val Accuracy for Epoch 1: 73.03 %
--------------------------------
Found better model...saving
Starting epoch 3
Train Loss for Epoch 2: 0.6677 
Train Accuracy for Epoch 2: 71.03 %
Val Loss for Epoch 2: 0.6564 
Val Accuracy for Epoch 2: 83.15 %
--------------------------------
Found better model...saving
Starting epoch 4
Train Loss for Epoch 3: 0.6494 
Train Accuracy for Epoch 3: 85.24 %
Val Loss for Epoch 3: 0.6337 
Val Accuracy for Epoch 3: 88.76 %
--------------------------------
Found better model...saving
Starting epoch 5
Train Loss for Epoch 4: 0.6322 
Train Acc

Val Loss for Epoch 19: 0.2539 
Val Accuracy for Epoch 19: 94.38 %
--------------------------------
Early Stop Count Increased
Found better model...saving
Starting epoch 21
Train Loss for Epoch 20: 0.1867 
Train Accuracy for Epoch 20: 97.49 %
Val Loss for Epoch 20: 0.2505 
Val Accuracy for Epoch 20: 93.26 %
--------------------------------
Early Stop Count Increased
Found better model...saving
Starting epoch 22
Train Loss for Epoch 21: 0.1633 
Train Accuracy for Epoch 21: 97.49 %
Val Loss for Epoch 21: 0.2648 
Val Accuracy for Epoch 21: 93.26 %
--------------------------------
Early Stop Count Increased
Starting epoch 23
Train Loss for Epoch 22: 0.1412 
Train Accuracy for Epoch 22: 98.05 %
Val Loss for Epoch 22: 0.2455 
Val Accuracy for Epoch 22: 89.89 %
--------------------------------
Early Stop Count Increased
Found better model...saving
Starting epoch 24
Train Loss for Epoch 23: 0.1638 
Train Accuracy for Epoch 23: 97.49 %
Val Loss for Epoch 23: 0.3131 
Val Accuracy for Epoch 23: 89

In [9]:
def evaluate():
    avg_acc = 0
    avg_f1 = 0
    avg_conf = np.zeros((2,2))
    
    # Preparation
    transformer = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5)),
    ])
    test_data_dir = './Prepared_Data/Test/'
    test_dataset = MyDataset(root_dir=test_data_dir, transform = transformer)

    for i in range(5):
        PATH = './Plan1_Models/best/model-fold-' + str(i) + '.pth'
        encoder = build_encoder()
        decoder = build_decoder()
        trained_net = Net(encoder,decoder)
        trained_net.load_state_dict(torch.load(PATH))
        trained_net.to(device)

        correct = 0
        total = 0
        y_true, y_pred = [], []
        testloader = torch.utils.data.DataLoader(
            test_dataset,
            batch_size = 50)

        y_pred = []
        y_true = []
        with torch.no_grad():
            # Iterate over the test data and generate predictions
            for i, data in enumerate(testloader, 0):

                # Get inputs
                inputs = data['images']
                targets = data['label']

                outputs = None
                for index, xbatch in enumerate(inputs):

                    xbatch = xbatch.float().to(device)

                    # Perform forward pass
                    output = sliding_window(xbatch, trained_net)
                    if index == 0:
                        outputs = output
                    else:
                        outputs = torch.vstack([outputs, output])

                # Compute loss
                ybatches = targets.to(device)
                outputs = torch.reshape(outputs, (ybatches.shape[0],NUM_CLASS))

                _, predicted = torch.max(outputs, 1)
                _, truth = torch.max(ybatches, 1)
                total += targets.size(0)
                correct += (predicted == truth).sum().item()

                y_pred.extend(predicted.cpu())
                y_true.extend(truth.cpu())

        # Print accuracy
        print("Accuracy", 100.0 * correct / total)
        f1 = f1_score(y_true, y_pred, labels=[0,1], average = 'binary')
        confusion_matr = confusion_matrix(y_true, y_pred, labels=[0,1])
        print("F1", f1)
        print(confusion_matr)

        avg_acc += 100.0 * correct / total
        avg_f1 += f1
        np.add(avg_conf, np.array(confusion_matr))

    avg_acc = avg_acc/5
    avg_f1 = avg_f1/5
    np.true_divide(avg_conf, 5)
    print("--------------Average------------")
    print('Avg Accuracy', avg_acc)
    print('Avg F1', avg_f1)
    print('Avg Confusion', avg_conf)


In [10]:
evaluate()

Accuracy 70.3125
F1 0.7710843373493976
[[13 19]
 [ 0 32]]
Accuracy 73.4375
F1 0.767123287671233
[[19 13]
 [ 4 28]]
Accuracy 75.0
F1 0.7948717948717949
[[17 15]
 [ 1 31]]
Accuracy 81.25
F1 0.8421052631578948
[[20 12]
 [ 0 32]]
Accuracy 87.5
F1 0.8823529411764706
[[26  6]
 [ 2 30]]
--------------Average------------
Avg Accuracy 77.5
Avg F1 0.8115075248453583
Avg Confusion [[0. 0.]
 [0. 0.]]
